# Revised Simplex Method


The revised simplex method may solve linear programs presented in 'standard form'. This solver is not very efficient and should rather be seen as a step by step demonstration of the revised simplex method. With good RAM it can handle a 1000 constraints or so.

--- 

Sample problem: 

        Max   2 x1 +   x2
        S.t.: 3 x1 + 4 x2 ≤ 6
              6 x1 +   x2 ≤ 3
                x1        ≥ 0
                       x2 ≥ 0

This problem has to variables and 2 constraints (+ positity constraints). The problem is solved below.


Some useful links:

http://www.trentu.ca/academic/math/bz/341-l06.pdf

https://www.math.ubc.ca/~anstee/math340/340exampleofduality.pdf

https://sites.google.com/site/yburda/math-340


In [1]:
## libraries 
import numpy as np
import scipy.linalg as la
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Sketch of Algorithm



step 1.
solve the system:
y = cb * inv(B)

step 2.
calculate:
enter = ( cn - y * An ) 

step 3.
chose variable with biggest coefficient (of 'enter') to enter basis, we call this vector a

step 4.
solve the system:
d = inv(B) * a

step 5.
Find largest t such that b - td >= 0, if there is no t, then problem is unbounded
otherweise at least one component of b/d = t equals zero. Note: division by zero will give inf, this is not a problem as we are looking for the argmin in the vector.

step 6.
Update the matrices:
Set value of entering variable at t and replace the values b of the basic variables by b - td. Replace the leaving column if B by the entering column and in the basis heading replace the leaving variable by the entering variable.

### Consider steps 1 and 3: use of elementary matrices to invert B

Bk = is basis after k iterations in the algorithm
Bk = Bk-1 * Ek 

Ek = identity matrix where pth collumn is replaced by d 

Factorize B into elementary matrices
Bk = B0 * E1 * E2 * ... Ek

this suggests thatthe system y * Bk = cb:
(((yE1)E2)...)Ek = cb

and Bk * d = a is:
E1(E2(...(Ek * d))) = a

### What about B0?

B0 is the initial B matrix before the first iteration. Usually this matrix corresponds to slack variables of a problem in standard form, in this case: B0 = I. If B0 is not the identity matrix we decompose it into permutations matrices and upper- and lower triangular matrices. Then we can use the same trick and use the product form of the inverse to compute the inverse of B to solve the system for the first iteration.

### Growing sequence of elementary matrices

As described above the first iteration creates elementary matrices from the LU decomposition of the initial B. As described by Chvatal memory can become an issue with large problems (this won't really matter in this python script) therefore I stored the matrices in 'compact form' as tuples in a list (I call this list 'inves'). The first entry is an integer representing the position of the column and the second entry is the d column of the elementary matrix. If the problem is very big, I can store only the relevant information of the eta matrix and 'recreate' it in the BTRAN and FTRAN function later without storing the matrix explicitly.

Lets assume the initial B is = 40x40. Before the first pivot I decompose B into P's, L's and U's. That means I start my list with 120 (40x40) matrices! Each consecutive iteration will add one elementary matrix for updating B, this sequence can become very long and cumbersome to calculate and may even take longer than inverting B explicitly. Therefore B is refactorized (decomposed again) every 20 iterations or so - Chvatal suggests this is more a rule of thumb than result from a proof.

If B0 = I, it is of course unnecessary to perform LU decomposition. Instead the algorithm can be initiated a tuple representing the identity matrix (uncomment tuple). Unlike in the text by Chvatal I store the inverse of the Eta matrix not the eta matrix itself.



In [2]:
class RevisedSimplex:
    
    def __init__(self, cn, An, b):
        ## as of now Basis will always be I, have to write function 
        ## that will catch arbitrary A and split it into An and B
        ## --> LU decomposition allows for this
        self.Initializer(cn, An, b)
        
    @classmethod
    def Initializer(cls, cn, An, b): 
        cls.An = An
        cls.B = (np.eye(len(An)))
        cls.b = b
        cls.cn = cn
        cls.cb = np.zeros(len(An))
        ## variable list constructor
        def lis(x, length):
            L = []
            for i in range(x):
                L.append("x" + str(i + length))
            return L
        cls.nonbasic = lis(len(An.T), 1)
        cls.basic = lis(len(An), 1 + len(cls.nonbasic))
        cls.Andic = dict(zip(cls.nonbasic, An.T[:,]))

        ## Decompose B!
        cls.inves = cls.TriangularFactorization(cls.B)
        
        ## Optimality Flag
        cls.optimal = False
        
        ## disabled trinagular fact. if initial basis is I
        '''
        one = np.matrix([1])
        zeros = np.zeros(len(An)-1).reshape(len(An)-1,1)
        cls.inves = [(0,np.vstack((one, zeros)))]
        '''
    def Enter(self):
         
        self.basic = RevisedSimplex.basic
        self.nonbasic = RevisedSimplex.nonbasic
        self.b = RevisedSimplex.b
        self.An = RevisedSimplex.An
        self.cb = RevisedSimplex.cb
        self.cn = RevisedSimplex.cn
        self.inves = RevisedSimplex.inves
        self.Andic = dict(zip(self.nonbasic, self.An.T[:,]))
        self.B = (np.eye(len(An)))
        
        def BTRAN(x):
            y = np.matrix(x) 
            for i in self.inves[::-1]:
                position, col = i
                invE = np.matrix(np.eye(len(An)))
                invE[:,position] = np.matrix(col)
                y = np.dot(y , invE)
            return y

        y = BTRAN(self.cb)
        
        enter = self.cn - np.dot(y, self.An)
        entering = int(np.argmax(enter, axis = 1)) 
        entering_variable = self.nonbasic[entering]
        optimal = np.all(enter <= 0)

        return entering, entering_variable, optimal 
    
    def Exit(self):
        entering, entering_variable, optimal = self.Enter(self)
        
        def FTRAN(x):
            d = x.reshape(len(x),-1)
            for i in self.inves:
                position, col = i
                invE = np.matrix(np.eye(len(An)))
                invE[:,position] = np.matrix(col)
                d = np.dot(invE , d)
            return d
        
        d = FTRAN(self.Andic[entering_variable])

        calculate_t = np.divide(self.b, d) 

        exiting = np.argmin(calculate_t)
        
        t = np.min(calculate_t)
        bnew = self.b - t*d 
        
        ## if checks if the problem is unbounded
        if np.any(bnew < 0) == True:
            raise Exception('This problem is unbounded!')
            
        bnew[bnew == 0] = t
        return entering, exiting, bnew, d, optimal
    
    @classmethod 
    def Update(self):
        ### --- Helper functions ---
        def changecollumn(matrix,position,d):
            E = matrix.copy()
            E[:,position] = np.matrix(d)
            return E
        def swap(A,B,i,j):
            TEMP_B = B.T[j].copy()
            B.T[j] = A.T[i]
            A.T[i] = TEMP_B
            return A,B
        
        entering, exiting, bnew, d, optimal = self.Exit(self)
        ## This terminates Algorithm, when the optimal dictionary is found
        RevisedSimplex.optimal = optimal
        
        # This file refactors the ETA file if it gets too long!
        if len(RevisedSimplex.inves) > len(RevisedSimplex.inves) + 20:
            self.refactorize()
        
        if RevisedSimplex.optimal == True:
            return
        
        print(entering, exiting)
        
        self.nonbasic[entering], self.basic[exiting] = self.basic[exiting], self.nonbasic[entering]
        RevisedSimplex.nonbasic = self.nonbasic
        RevisedSimplex.basic = self.basic
        RevisedSimplex.cn, RevisedSimplex.cb = swap(self.cn,self.cb,entering,exiting)
        RevisedSimplex.An[:,entering] = self.B[:,exiting] 
        RevisedSimplex.b = bnew 
        
        def Etainverse(matrix,position,a):
            E = matrix
            E[:,position] = a.reshape(-1,1)
            test = E[position,position]
            E[:,position] = -E[:,position]/test
            E[position,position] = 1/test
            Epack = tuple((position, E[:,position]))
            return Epack
        
        Epack = Etainverse(np.matrix(np.eye(len(An))), exiting, d)
        self.inves.append(Epack)
        RevisedSimplex.inves = self.inves


    def Pivot(self):
        # counts the number of pivtos 
        count = 0
        while RevisedSimplex.optimal != True:
            self.Update()
            count += 1
        return count
    
    def solve(self):
        count = self.Pivot()
        if RevisedSimplex.optimal == True:
            z = self.maximum()
            print("The current dictionary is optimal! The number of iterations to solve was:", count) 
            print("maximum value is:", z)

    
    ### --- Helper functions --- ###
    
    def tabulate():
        # will write function that puts results in table
        # also want to show slack per variable
        # small table
        pass
    
    def cycling():
        # cycling is rare but is guaranteed to never occur under "bland's rule"
        # -> chose smallest subscript variable if there is a choice 
        # do this later, cycling almost never occurs
        pass
    
    def maximum(self):
        if RevisedSimplex.optimal == True: 
            z = np.dot(RevisedSimplex.cb,  RevisedSimplex.b)
            return z
        
    @staticmethod
    def TriangularFactorization(X):
        (P, L, U) = la.lu(X) #B matrix does not have to be I :-)
        
        def InvEtafact(matrix):
            d = np.matrix(matrix)
            Ilist = []
            for i in range(len(matrix)):
                col = d[:,i]
                I = np.matrix(np.eye(len(matrix)))
                I[:,i] = col
                test = I[i,i]
                I[:,i] = -I[:,i]/test
                I[i,i] = 1/test
                Ipack = tuple((i, I[:,i]))
                Ilist.append(Ipack)
            return Ilist
        l = InvEtafact(L)
        p = InvEtafact(P)
        u = InvEtafact(U)
        
        def createETA(p,l,u):
            c = [j for i in zip(p,l) for j in i]
            Um = u[::-1]
            new = c + Um
            return new
        initiate = createETA(p,l,u)
        return initiate
    
    @classmethod
    def refactorize(self):
        ## This function will refactorize B the current B when the ETA file
        ## or "inves" gets too long, maybe after 20 iterations? 
        def reconstruct():
            y = np.eye(len(RevisedSimplex.An))
            for i in RevisedSimplex.inves:
                pos, col = i 
                I = np.matrix(np.eye(len(RevisedSimplex.An)))
                # I[:,i] = col
                test = I[pos,pos]
                I[:,pos] = -I[:,pos]/test
                I[pos,pos] = 1/test
                y = np.dot(y , I)
            return y
        newB = reconstruct()
        RevisedSimplex.inves = RevisedSimplex.TriangularFactorization(newB)

In [3]:
## Solves example problem above

An = np.array([[3,4],[6,1]])
cn = np.matrix([2,1])
b = np.array([[6],[3]])


wow = RevisedSimplex(cn,An,b).solve()

0 1
1 0
The current dictionary is optimal! The number of iterations to solve was: 3
maximum value is: [[1.85714286]]


In [ ]:
## Larger scale test
## careful, only test if you have sufficient RAM! 

size = 1000
#how many constraints
length = 1000

An = np.random.randint(2, 3, (length,size))
cn = np.random.randint(1, 2, (1,size))
b = np.random.randint(50, 70, (length,1))

wow = RevisedSimplex(cn,An,b).solve()